In [2]:
import pandas as pd

In [ ]:
zmb51 = pd.read_excel(r'C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\Py\PowerBI\Novo\zmb51_historico.xlsx')


In [17]:
df_zstok_new = pd.read_excel(r"C:\Users\fsp_adolpho.salvador\Desktop\Konica Minolta\Desktop Cloud - Documentos\Desktop\zstok.XLSX")

In [18]:
df_zstok_new.columns

Index(['Tipo de material', 'Material', 'Descrição', 'UMB', 'Cen.', 'Dep.',
       'Pos.dpst.', 'Utiliz.livre', 'Contr.qualid.', 'Transf.dep.',
       'Transfer.(centro)', 'Bloqueado', 'Estoque Total', 'Reservado',
       'Remessa->Cliente', 'SC', 'Status todos centros', 'SV', 'Status vendas',
       'Valor Total', 'Valor Unitario'],
      dtype='object')

In [7]:


# Converter a coluna 'Qtd.' para numérico e tratar valores nulos como zero
zmb51['Qtd.'] = pd.to_numeric(zmb51['Qtd.  UM registro'], errors='coerce').fillna(0)

# Filtrar apenas os registros com 'CONCEITO DE PESQUISA_2' igual a 'Indireto SP'
df_filtrado = zmb51[(zmb51['CONCEITO DE PESQUISA_2'] == 'Indireto SP') & (zmb51['Tipo de Material'] == 'PCCC')]


# Agrupar por SKU ('Material') e somar as quantidades de saída
sku_saida = df_filtrado.groupby('Material')['Qtd.  UM registro'].sum().reset_index()

# Renomear colunas para clareza
sku_saida.columns = ['SKU', 'Total de Saídas']

# Exibir o resultado
print(sku_saida)


          SKU  Total de Saídas
0     4047201              2.0
1     4062201             11.0
2     4062301              2.0
3     4062401              2.0
4     4062501              2.0
..        ...              ...
333   S-6710G              7.0
334   S-6711G              4.0
335   S-6712G              2.0
336   S-7198U              4.0
337  S-7265UA              5.0

[338 rows x 2 columns]


In [8]:


# Converter a coluna 'Qtd.' para numérico e tratar valores nulos como zero
zmb51['Qtd.'] = pd.to_numeric(zmb51['Qtd.'], errors='coerce').fillna(0)

# Filtrar para 'Indireto SP' e 'Tipo de Material' igual a 'PCCC'
df_filtrado = zmb51[(zmb51['CONCEITO DE PESQUISA_2'] == 'Indireto SP') & (zmb51['Tipo de Material'] == 'PCCC')]

# Converter a coluna de datas para o formato de data, para organizar por mês e ano
df_filtrado['Data de lançamento'] = pd.to_datetime(df_filtrado['Data de lançamento'], errors='coerce')

# Agrupar os dados por SKU, mês e ano para calcular o consumo mensal
df_filtrado['AnoMes'] = df_filtrado['Data de lançamento'].dt.to_period('M')
consumo_mensal = df_filtrado.groupby(['Material', 'AnoMes'])['Qtd.'].sum().reset_index()

# Calcular média, desvio padrão e coeficiente de variação (CV) para cada SKU
previsibilidade = consumo_mensal.groupby('Material')['Qtd.'].agg(['mean', 'std']).reset_index()
previsibilidade['CV'] = (previsibilidade['std'] / previsibilidade['mean']) * 100

# Classificar previsibilidade com base no CV
def classificar_previsibilidade(cv):
    if cv <= 20:
        return 'X'  # Alta previsibilidade
    elif 20 < cv <= 50:
        return 'Y'  # Previsibilidade moderada
    else:
        return 'Z'  # Baixa previsibilidade

previsibilidade['Classe Previsibilidade'] = previsibilidade['CV'].apply(classificar_previsibilidade)

# Combinar resultados com total de saídas por SKU
total_saida = df_filtrado.groupby('Material')['Qtd.'].sum().reset_index()
total_saida.columns = ['SKU', 'Total de Saídas']

# Unir total de saídas e previsibilidade em um único DataFrame
resultado = pd.merge(total_saida, previsibilidade[['Material', 'CV', 'Classe Previsibilidade']], left_on='SKU', right_on='Material').drop(columns='Material')

# Exibir o resultado
resultado


C:\Users\fsp_adolpho.salvador\AppData\Local\Temp\ipykernel_19756\1106993788.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['Data de lançamento'] = pd.to_datetime(df_filtrado['Data de lançamento'], errors='coerce')
C:\Users\fsp_adolpho.salvador\AppData\Local\Temp\ipykernel_19756\1106993788.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['AnoMes'] = df_filtrado['Data de lançamento'].dt.to_period('M')


,SKU,Total de Saídas,CV,Classe Previsibilidade
0,4047201,2.0,NaN,Z
1,4062201,11.0,115.708382,Z
2,4062301,2.0,NaN,Z
3,4062401,2.0,NaN,Z
4,4062501,2.0,NaN,Z
...,...,...,...,...
333,S-6710G,7.0,NaN,Z
334,S-6711G,4.0,NaN,Z
335,S-6712G,2.0,NaN,Z
336,S-7198U,4.0,NaN,Z


In [9]:
resultado.to_excel('resultado_XYZ.xlsx')

In [19]:
# As per the previous analysis, let's confirm the columns to perform the ABC analysis using "Valor Unitario" and "Estoque Total" columns

# Convert "Valor Unitario" to numeric and handle any non-numeric values as zero
df_zstok_new['Valor Unitario'] = pd.to_numeric(df_zstok_new['Valor Unitario'], errors='coerce').fillna(0)

# Convert "Estoque Total" to numeric and handle any non-numeric values as zero
df_zstok_new['Estoque Total'] = pd.to_numeric(df_zstok_new['Estoque Total'], errors='coerce').fillna(0)

# Calculate the annual consumption value for each SKU
df_zstok_new['Valor de Consumo Anual'] = df_zstok_new['Valor Unitario'] * df_zstok_new['Estoque Total']

# Filter out items with zero "Valor de Consumo Anual" as they wouldn't be relevant for ABC analysis
df_zstok_new = df_zstok_new[df_zstok_new['Valor de Consumo Anual'] > 0]

# Sort by "Valor de Consumo Anual" in descending order
df_zstok_new = df_zstok_new.sort_values(by='Valor de Consumo Anual', ascending=False)

# Calculate the total consumption value for ABC percentage calculations
total_consumo_anual = df_zstok_new['Valor de Consumo Anual'].sum()

# Calculate the percentage and cumulative percentage of each SKU
df_zstok_new['Participação Percentual'] = (df_zstok_new['Valor de Consumo Anual'] / total_consumo_anual) * 100
df_zstok_new['Participação Acumulada'] = df_zstok_new['Participação Percentual'].cumsum()

# Classify into A, B, or C based on cumulative percentage
def classificar_abc(participacao_acumulada):
    if participacao_acumulada <= 80:
        return 'A'
    elif participacao_acumulada <= 95:
        return 'B'
    else:
        return 'C'

df_zstok_new['Classe ABC'] = df_zstok_new['Participação Acumulada'].apply(classificar_abc)

# Select relevant columns for the final ABC classification result
df_abc_result = df_zstok_new[['Material', 'Valor Unitario', 'Estoque Total', 'Valor de Consumo Anual', 
                              'Participação Percentual', 'Participação Acumulada', 'Classe ABC']]

# Saving the ABC analysis result to an Excel file for export

df_abc_result.to_excel('resultado_abc_analysis_new.xlsx')